#**Машинное обучение ИБ-2024**

#**Домашнее задание 1.**
#Регрессия, KNN, LinearRegression.

В данной домашней работе мы будем строить модели для предсказания цены квартиры в России. Ниже приведено описание некоторых колонок набора данных.

date - дата публикации объявления

price - цена в рублях

level- этаж, на котором находится квартира

levels - количество этажей в квартире

rooms - количество комнат в квартире. Если значение -1, то квартира считается апартаментами.

area - площадь квартиры.

kitchen_area - площадь кухни.

geo_lat - Latitude

geo_lon - Longitude

building_type - материал застройки. 0 - Don't know. 1 - Other. 2 - Panel. 3 - Monolithic. 4 - Brick. 5 - Blocky. 6 - Wooden

#Часть 0. Начало работы

Для начала работы с данными импортируем библиотеки, которые понадобятся в данном задании.

In [1]:
import math
import pandas as pd
import numpy as np
import matplotlib as plt
import sklearn
import seaborn as sns

Загрузим библиотеку folium для отображения данных на карте по координатам.

In [2]:
!pip install folium

Распакуем наши данные из архива.

Загрузим данные из csv файла в датафрейм.

In [3]:
df = pd.read_csv('input_data.csv', sep=';')

Отобразим на карте координаты наших построек.

# Часть 1. Подготовим данные для обработки моделями машинного обучения.

**0.5 Балл**. География наших наблюдений в наборе данных крайне большая. Однако мы знаем, что стоимость квартир в Москве и Санкт-Петербурге намного выше, чем в среднем по России. Давайте сделаем признаки, который показывают, находится ли квартира в 20 килиметрах от центра Москвы или находится ли квартира в 20 килиметрах от центра Санкт-Петербурга.

Создайте два признака is_Moscow и is_Saint_Peterburg. Для нахождения расстояния по координатам используйте функцию haversine_distance.

In [4]:
def deg2rad(deg):
  return deg * 3.1415 / 180

def haversine_distance(lat1, lon1, lat2, lon2):
  R = 6371
  dLat = deg2rad(lat2-lat1)
  dLon = deg2rad(lon2-lon1);
  a = math.sin(dLat/2) * math.sin(dLat/2) +\
    math.cos(deg2rad(lat1)) * math.cos(deg2rad(lat2)) *\
    math.sin(dLon/2) * math.sin(dLon/2)

  c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a));
  d = R * c
  return d

In [5]:
def is_Moscow(row):
  lat=row['geo_lat']
  lon=row['geo_lon']
  dist=haversine_distance(lat, lon, lat2=55.7522, lon2=37.6156)
  return dist <= 20

def is_Saint_Peterburg(row):
  lat=row['geo_lat']
  lon=row['geo_lon']
  dist=haversine_distance(lat, lon, lat2=59.9386, lon2=30.3141)
  return dist <= 20

df['is_Moscow']=          df.apply(is_Moscow, axis=1)
df['is_Saint_Peterburg']= df.apply(is_Saint_Peterburg, axis=1)

**0.5 Балла**. В нашем наборе данных есть признаки, которые мы теоретически можем использовать, например postal_code, но мы это будем делать в рамках домашней работы очень-очень долго. Поэтому предлагается удалить ненужные признаки из датафрейма.

Удалим geo_lat,	geo_lon,	object_type,	postal_code,	street_id,	id_region,	house_id.

In [7]:
df_clean=df.drop(['geo_lat', 'geo_lon', 'object_type', 'postal_code', 'street_id', 'id_region', 'house_id'], axis=1)

df_clean['building_type']=df_clean['building_type'].map({
    0:"Unknown material",
    1:"Other",
    2:"Panel",
    3:"Monolithic",
    4:"Brick",
    5:"Blocky",
    6:"Wooden",
})

**0.5 Балл**. Для начала Вам предлагается проанализировать Ваши оставшиеся признаки (колонки) в наборе данных. Какие колонки категориальные? Какие числовые?

Категориальные: building_type, is_Moscow, is_Saint_Peterburg

Числовые: date, price, level, levels, area, kitchen_area, rooms

Давайте закодируем категориальные признаки с помощью OneHot-Encoding. https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html

In [8]:
from sklearn.preprocessing import OneHotEncoder
onehotencoder = OneHotEncoder(sparse_output = False)


encoded_df = pd.DataFrame(onehotencoder.fit_transform(df_clean[['building_type']]))
encoded_df.columns = onehotencoder.get_feature_names_out()

In [9]:
df_onehot=df_clean.join(encoded_df)
df_onehot.drop('building_type', axis=1, inplace=True)
df_onehot['is_Moscow']=df_onehot['is_Moscow'].astype(int)
df_onehot['is_Saint_Peterburg']=df_onehot['is_Saint_Peterburg'].astype(int)

**0.5 Балл**. Поработаем с числовыми признаками:


1.   Добавьте в ваш датасет два признака: количество дней со дня первого наблюдения (разница между датами объявлений). Возможно, для предсказания цены не так важен этаж, как важно отношение этажа квартиры на количество этажей в доме, добавьте этот признак. После добавления нового признака колонку date можно удалить.
2.   Числовые признаки могут иметь разные порядки. Давайте отнормируем числовые признаки с помощью StandartScaller https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html.



In [10]:
df_onehot['levels_ratio'] = df_onehot['level'] / df_onehot['levels']

In [11]:
df_onehot['date']=pd.to_datetime(df_onehot['date'])

In [12]:
start_date=df_onehot.date.min()

In [13]:
df_onehot['days_from_start'] = (df_onehot['date'] - start_date).dt.days

In [14]:
df_onehot.drop('date',axis=1,inplace=True)

In [23]:
df_onehot.describe()

,price,level,levels,rooms,area,kitchen_area,is_Moscow,is_Saint_Peterburg,building_type_Blocky,building_type_Brick,building_type_Monolithic,building_type_Other,building_type_Panel,building_type_Unknown material,building_type_Wooden,levels_ratio,days_from_start
count,6.616400e+04,66164.000000,66164.000000,66164.000000,66164.000000,66164.000000,66164.000000,66164.000000,66164.000000,66164.000000,66164.000000,66164.000000,66164.000000,66164.000000,66164.000000,6.616300e+04,66164.000000
mean,1.562671e+07,6.348800,11.422163,1.788525,54.742582,-8.859225,0.100462,0.062708,0.026646,0.252539,0.145170,0.023185,0.189227,0.358367,0.004867,inf,2.967218
std,2.470915e+09,5.440571,7.369376,1.183349,28.269475,40.508144,0.300618,0.242439,0.161048,0.434472,0.352274,0.150491,0.391692,0.479524,0.069592,NaN,1.813226
min,1.000000e+00,0.000000,0.000000,-1.000000,5.300000,-100.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000
25%,2.200000e+06,2.000000,5.000000,1.000000,37.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.333333e-01,1.000000
50%,3.469500e+06,5.000000,9.000000,2.000000,48.300000,7.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000e-01,3.000000
75%,5.893228e+06,9.000000,16.000000,3.000000,65.000000,11.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,8.333333e-01,5.000000
max,6.355524e+11,50.000000,50.000000,8.000000,497.000000,160.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,inf,6.000000


In [27]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

df_2=df_onehot[df_onehot.levels != 0].copy()
df_2 = pd.DataFrame(StandardScaler().fit_transform(df_2),columns = df_2.columns)

2 Балла. Реализуйте класс KNNRegressor, который должен делать регрессию методом k ближайших соседей.

In [102]:
from scipy.spatial import distance

class KNNRegressor:
    def __init__(self, n_neighbors=5, metric='euclidean'):
        self.n_neighbors = n_neighbors
        self.metric = metric
        self.X_train = None
        self.y_train = None

    def fit(self, X_train, y_train):
        self.X_train =  X_train
        self.y_train = y_train

    def _euclidean_distances(self, x_test_i):
        try:
          result =   np.sqrt(np.sum((self.X_train - x_test_i) ** 2, axis=1))
          return result
        except Exception as e:
          print(x_test_i)
          print(e)
          raise ValueError()

    def _make_prediction(self, x_test_i):
        distances = self._euclidean_distances(x_test_i)
        k_nearest_indexes = np.argsort(distances)[:self.n_neighbors]
        targets = self.y_train[k_nearest_indexes]

        return np.mean(targets)

    def predict(self, X_test):
        return np.array([self._make_prediction(x) for x in X_test.values])

In [103]:
knn_clf = KNNRegressor(n_neighbors=5)
knn_clf.fit(X_train, y_train)
knn_clf_pred_res = knn_clf.predict(X_test)

**3 Балла**. Реализуйте класс LinearRegression, поддерживающий обучение градиентными спусками SGD, Momentum, AdaGrad. Используйте градиент для оптимизации функции потерь MSE.

In [29]:
import numpy as np

class LinearRegression:
    def __init__(self, learning_rate=0.01, optimization='SGD', epsilon=1e-8, decay_rate=0.9, max_iter=1000):
        self.learning_rate = learning_rate
        self.optimization = optimization
        self.epsilon = epsilon
        self.decay_rate = decay_rate
        self.max_iter = max_iter
        self.weights = None
        self.bias = None

    def fit(self, X, y):
        X = np.array(X)
        y = np.array(y)
        n_samples, n_features = X.shape
        self.weights = np.zeros(n_features)
        self.bias = 0

        # Инициализация для Momentum
        velocity_w = np.zeros(n_features)
        velocity_b = 0

        # Инициализация для AdaGrad
        grad_squared_w = np.zeros(n_features)
        grad_squared_b = 0

        for _ in range(self.max_iter):
            # Предсказания
            y_pred = np.dot(X, self.weights) + self.bias

            # Вычисляем градиенты
            dw = (1 / n_samples) * np.dot(X.T, (y_pred - y))
            db = (1 / n_samples) * np.sum(y_pred - y)

            if self.optimization == 'SGD':
                # Обновление параметров при использовании SGD
                self.weights -= self.learning_rate * dw
                self.bias -= self.learning_rate * db

            elif self.optimization == 'Momentum':
                # Обновление параметров с использованием Momentum
                velocity_w = self.decay_rate * velocity_w + (1 - self.decay_rate) * dw
                velocity_b = self.decay_rate * velocity_b + (1 - self.decay_rate) * db
                self.weights -= self.learning_rate * velocity_w
                self.bias -= self.learning_rate * velocity_b

            elif self.optimization == 'AdaGrad':
                # Обновление параметров с использованием AdaGrad
                grad_squared_w += dw ** 2
                grad_squared_b += db ** 2
                self.weights -= (self.learning_rate / (np.sqrt(grad_squared_w) + self.epsilon)) * dw
                self.bias -= (self.learning_rate / (np.sqrt(grad_squared_b) + self.epsilon)) * db

            else:
                raise ValueError("Unknown optimization method: {}".format(self.optimization))

    def predict(self, X):
        X = np.array(X)
        return np.dot(X, self.weights) + self.bias

# Часть 2. Эксперименты с моделями машинного обучения.

**3 Балла**. Проведите эксперименты с написанными Вами методами машинного обучения. Выделите обучающую и тестовую выборки в отношении 0,8 и 0,2 соответственно. Измерьте ошибку MSE, MAE, RMSE. Заиспользуйте методы KNNRegressor и LinearRegression из библиотеки sklearn, сравните качество Ваших решений и библиотечных.

In [99]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression as SKLinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error

y = df_2.price
X=df_2.drop(columns=['price'])
# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
X_train.reset_index(inplace=True)
X_test.reset_index(inplace=True)

y_train = np.array(y_train)
y_test = np.array(y_test)

### KNN

In [110]:
knn_regressor = KNeighborsRegressor(n_neighbors=5)
knn_regressor.fit(X_train, y_train)
y_pred_knn = knn_regressor.predict(X_test)

In [111]:
y_pred_knn

array([-0.00264707, -0.00387759, -0.00550692, ..., -0.0052781 ,
       -0.0053637 , -0.00448538])

In [100]:
knn_clf = KNNRegressor(n_neighbors=5)
knn_clf.fit(X_train, y_train)
knn_clf_pred_res = knn_clf.predict(X_test)

AttributeError: 'KNNRegressor' object has no attribute 'regression'

### LinearRegression

In [31]:
sklinear_regressor = SKLinearRegression()
sklinear_regressor.fit(X_train, y_train)
y_pred_sklinear = sklinear_regressor.predict(X_test)

In [33]:
my_linear_regressor = LinearRegression(learning_rate=0.01, max_iter=1000)
my_linear_regressor.fit(X_train, y_train)
y_pred_my_linear = my_linear_regressor.predict(X_test)

In [105]:
y_pred_knn

array([[ 2.15212000e+04, -2.64707193e-03],
       [ 4.99658000e+04, -3.87758639e-03],
       [ 5.35994000e+04, -5.50691946e-03],
       ...,
       [ 5.64880000e+03, -5.27810252e-03],
       [ 5.04062000e+04, -5.36369640e-03],
       [ 6.25172000e+04, -4.48537768e-03]])

In [112]:
# Оценка качества моделей
def print_metrics(y_true, y_pred, model_name):
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    print(f"Метрики для модели {model_name}:")
    print(f"MSE: {mse:.4f}")
    print(f"MAE: {mae:.4f}")
    print(f"RMSE: {rmse:.4f}\n")

# Выводим метрики
print_metrics(y_test, y_pred_sklinear, "Библиотечная линейная регрессия")
print_metrics(y_test, y_pred_knn, "Библиотечный KNN Regressor")
print_metrics(y_test, y_pred_my_linear, "Собственная линейная регрессия")
print_metrics(y_test, knn_clf_pred_res, "Собственная KNN Regressor")

Метрики для модели Библиотечная линейная регрессия:
MSE: 0.0006
MAE: 0.0180
RMSE: 0.0251

Метрики для модели Библиотечный KNN Regressor:
MSE: 0.0001
MAE: 0.0017
RMSE: 0.0073

Метрики для модели Собственная линейная регрессия:
MSE: 0.0005
MAE: 0.0164
RMSE: 0.0229

Метрики для модели Собственная KNN Regressor:
MSE: 0.0001
MAE: 0.0017
RMSE: 0.0073

